In [9]:
import os
import conllu
#from conllu import parse
import spacy
from spacy.language import Language
spacy.prefer_gpu()

True

In [10]:
nlp = spacy.load("grc_proiel_trf")

In [2]:
# if using vectors
!python -m spacy init vectors grc ../assets/grc_floret_cbow_nn2_xn10_b200k_dim300.floret ../vectors/large --mode floret

ℹ Creating blank nlp object for language 'grc'
[2023-05-03 10:33:32,020] [INFO] Reading vectors from ../assets/grc_floret_cbow_nn2_xn10_b200k_dim300.floret
200000it [00:07, 28266.74it/s]
[2023-05-03 10:33:39,098] [INFO] Loaded vectors from ../assets/grc_floret_cbow_nn2_xn10_b200k_dim300.floret
✔ Successfully converted 200000 vectors
✔ Saved nlp object with vectors to output directory. You can now use
the path to it in your config as the 'vectors' setting in [initialize].
/root/Projects/Atlomy/git/greCy_ATLOMY/vectors/large


In [22]:
# if making senter
!python -m spacy train ../configs/senter_vec.cfg --output ../training/senter/large --gpu-id 0 --paths.train ../corpus/train/grc_perseus-ud-train.spacy --paths.dev ../corpus/dev/grc_perseus-ud-dev.spacy --paths.vectors ../vectors/large --nlp.lang=grc

✔ Created output directory: ../training/senter/large
ℹ Saving to output directory: ../training/senter/large
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-05-03 11:03:36,280] [INFO] Set up nlp object from config
[2023-05-03 11:03:36,288] [INFO] Pipeline: ['senter']
[2023-05-03 11:03:36,290] [INFO] Created vocabulary
[2023-05-03 11:03:36,685] [INFO] Added vectors: ../vectors/large
[2023-05-03 11:03:37,079] [INFO] Finished initializing nlp object
[2023-05-03 11:03:39,483] [INFO] Initialized pipeline components: ['senter']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['senter']
ℹ Initial learn rate: 0.001
E    #       LOSS SENTER  SENTS_F  SCORE 
---  ------  -----------  -------  ------
  0       0        73.00     0.20    0.00
  0     200      2920.38    93.06    0.93
  0     400       389.75    93.99    0.94
  0     600       281.08    95.59    0.96
  0     800     

In [24]:
!python -m spacy train ../configs/lemmatizer_vec.cfg --output ../training/lemmatizer/large --gpu-id 0 --paths.train ../corpus/train/lemma_train --paths.dev ../corpus/dev/lemma_dev --paths.vectors ../vectors/large --nlp.lang=grc


ℹ Saving to output directory: ../training/lemmatizer/large
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-05-03 11:10:53,265] [INFO] Set up nlp object from config
[2023-05-03 11:10:53,272] [INFO] Pipeline: ['lemmatizer']
[2023-05-03 11:10:53,275] [INFO] Created vocabulary
[2023-05-03 11:10:53,669] [INFO] Added vectors: ../vectors/large
[2023-05-03 11:10:54,064] [INFO] Finished initializing nlp object
[2023-05-03 11:11:07,586] [INFO] Initialized pipeline components: ['lemmatizer']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['lemmatizer']
ℹ Initial learn rate: 0.001
E    #       LOSS LEMMA...  LEMMA_ACC  SCORE 
---  ------  -------------  ---------  ------
  0       0         141.00      25.77    0.26
  0     200       30759.78      60.75    0.61
  0     400       24842.97      70.83    0.71
  0     600       16802.02      74.58    0.75
  0     800       16110.00   

In [26]:
!python -m spacy train ../configs/transformer.cfg --output ../training/transformer/proiel/assembled --gpu-id 0 --paths.train ../corpus/train/grc_proiel-ud-train.spacy --paths.dev ../corpus/dev/grc_proiel-ud-dev.spacy --nlp.lang=grc


✔ Created output directory:
../training/transformer/proiel/assembled
ℹ Saving to output directory:
../training/transformer/proiel/assembled
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-05-03 12:32:35,402] [INFO] Set up nlp object from config
[2023-05-03 12:32:35,411] [INFO] Pipeline: ['transformer', 'morphologizer', 'tagger', 'parser', 'senter', 'lemmatizer', 'attribute_ruler']
[2023-05-03 12:32:35,416] [INFO] Created vocabulary
[2023-05-03 12:32:35,770] [INFO] Added vectors: ../vectors/large
[2023-05-03 12:32:36,019] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at Jacobo/aristoBERTo were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expect

In [6]:
# remove accents definition
import unicodedata
def clean_text(text: str) -> str:
    #Cleans the given text by stripping accents and lowercasing.
    try:
        non_accent_characters = [
        char for char in unicodedata.normalize('NFD', text)
        if unicodedata.category(char) != 'Mn'
        or char == '̓'  # Greek coronis
        ]
    # str.lower() works for unicode characters
        return ''.join(non_accent_characters).lower()
    except TypeError:
        return text


In [32]:
# Normalization to NFKD
import unicodedata
def normalize(text):
    return unicodedata.normalize('NFKD', text)
# # load files from folder
for file in os.listdir("../assets/Lemmatization_training_files"):
    if file.endswith(".conllu"):
        print(file)
        data = open("../assets/Lemmatization_training_files/" + file, "r", encoding="utf-8")
        annotations = data.read()
        sentences = conllu.parse(annotations)
        with open("../assets/Lemmatization_training_files/lemma_train_NFKD/" + "{0}_NFKD.conllu".format(file[:-7]), "w", encoding="utf-8") as data:
            for sentence in sentences:
                for token in sentence:
                    if token["form"] == '':
                        print ('Empty form:', token["form"], token["lemma"], token["id"], token["upos"], sentence)
                        token["form"] = token["lemma"]
                    #print(clean_text(sentence))
                    if token["lemma"] == '':
                        print ('Empty lemma:', token["form"], token["lemma"], token["id"], token["upos"], sentence)
                        token["lemma"] = token["form"]
                    if token["id"] == '':
                        print ('Empty id:', token["form"], token["lemma"], token["id"], token["upos"], sentence)
                        token["id"] = 'UNK'
                    if token["upos"] == '':
                        print ('Empty upos:', token["form"], token["lemma"], token["id"], token["upos"], sentence)
                        token["upos"] = 'UNK'
                    token["form"] = normalize(token["form"])
                    token["lemma"] = normalize(token["lemma"])
                data.write(sentence.serialize())
        data.close()

xenophon_i_train.conllu
herodotus_train.conllu
galen_dev.conllu
aeschylus_i_train.conllu
plato_iii_train.conllu
plato_iii_dev.conllu
xenophon_ii_train.conllu
galen_train.conllu
aeschylus_ii_train.conllu
euripides_i_train.conllu
aeschylus_i_dev.conllu
xenophon_ii_dev_spaced.conllu
plato_ii_train_spaced.conllu
Empty form:  > 11 PUNCT TokenList<ΘΕΟ., τὰ, ποῖα, δή, ;, ΣΩ., "", #, Σωκράτης, "", , ΣΩ., σοφιστήν, ,, πολιτικόν, ,, φιλόσοφον, ., ΘΕΟ., τί, δὲ, μάλιστα, καὶ, τὸ, ποῖόν, τι, περὶ, αὐτῶν, διαπορηθεὶς, ἐρέσθαι, διενοήθης, ;, ΣΩ., τόδε, ·, πότερον, ἓν, πάντα, ταῦτα, ἐνόμιζον, ἢ, δύο, ,, ἢ, καθάπερ, τὰ, ὀνόματα, τρία, ,, τρία, καὶ, τὰ, γένη, διαιρούμενοι, καθʼ, ἓν, ὄνομα, γένος, ἑκάστῳ, προσῆπτον, ;, ΘΕΟ., ἀλλʼ, οὐδείς, ,, ὡς, ἐγᾦμαι, ,, φθόνος, αὐτῷ, διελθεῖν, αὐτά, ·, ἢ, πῶς, ,, ὦ, ξένε, ,, λέγωμεν, ;, ΞΕ., #, Ξένος, "", , ΞΕ., οὕτως, ,, ὦ, Θεόδωρε, ., φθόνος, μὲν, γὰρ, οὐδεὶς, οὐδὲ, χαλεπὸν, εἰπεῖν, ὅτι, γε, τρίʼ, ἡγοῦντο, ·, καθʼ, ἕκαστον, μὴν, διορίσασθαι, σαφῶς, τί, ποτʼ, ἔστ

In [33]:
# convert conllu to spacy Lemmatization_training_files with NFKD normalization
!python -m spacy convert ../assets/Lemmatization_training_files/lemma_train_NFKD/ ../assets/Lemmatization_training_files/lemma_train_NFKD/ -c conllu -m --n-sents 10 --merge-subtokens

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (49 documents):
../assets/Lemmatization_training_files/lemma_train_NFKD/aeschylus_i_dev_NFKD.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (185 documents):
../assets/Lemmatization_training_files/lemma_train_NFKD/aeschylus_i_train_NFKD.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (33 documents):
../assets/Lemmatization_training_files/lemma_train_NFKD/aeschylus_ii_dev_NFKD.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (136 documents):
../assets/Lemmatization_training_files/lemma_train_NFKD/aeschylus_ii_train

In [34]:
# Normalization to NFKC
import unicodedata
def normalize(text):
    return unicodedata.normalize('NFKC', text)
# # load files from folder
for file in os.listdir("../assets/Lemmatization_training_files"):
    if file.endswith(".conllu"):
        print(file)
        data = open("../assets/Lemmatization_training_files/" + file, "r", encoding="utf-8")
        annotations = data.read()
        sentences = conllu.parse(annotations)
        with open("../assets/Lemmatization_training_files/lemma_train_NFKC/" + "{0}_NFKC.conllu".format(file[:-7]), "w", encoding="utf-8") as data:
            for sentence in sentences:
                for token in sentence:
                    if token["form"] == '':
                        print ('Empty form:', token["form"], token["lemma"], token["id"], token["upos"], sentence)
                        token["form"] = token["lemma"]
                    #print(clean_text(sentence))
                    if token["lemma"] == '':
                        print ('Empty lemma:', token["form"], token["lemma"], token["id"], token["upos"], sentence)
                        token["lemma"] = token["form"]
                    if token["id"] == '':
                        print ('Empty id:', token["form"], token["lemma"], token["id"], token["upos"], sentence)
                        token["id"] = 'UNK'
                    if token["upos"] == '':
                        print ('Empty upos:', token["form"], token["lemma"], token["id"], token["upos"], sentence)
                        token["upos"] = 'UNK'
                    token["form"] = normalize(token["form"])
                    token["lemma"] = normalize(token["lemma"])
                data.write(sentence.serialize())
        data.close()

xenophon_i_train.conllu
herodotus_train.conllu
galen_dev.conllu
aeschylus_i_train.conllu
plato_iii_train.conllu
plato_iii_dev.conllu
xenophon_ii_train.conllu
galen_train.conllu
aeschylus_ii_train.conllu
euripides_i_train.conllu
aeschylus_i_dev.conllu
xenophon_ii_dev_spaced.conllu
plato_ii_train_spaced.conllu
Empty form:  > 11 PUNCT TokenList<ΘΕΟ., τὰ, ποῖα, δή, ;, ΣΩ., "", #, Σωκράτης, "", , ΣΩ., σοφιστήν, ,, πολιτικόν, ,, φιλόσοφον, ., ΘΕΟ., τί, δὲ, μάλιστα, καὶ, τὸ, ποῖόν, τι, περὶ, αὐτῶν, διαπορηθεὶς, ἐρέσθαι, διενοήθης, ;, ΣΩ., τόδε, ·, πότερον, ἓν, πάντα, ταῦτα, ἐνόμιζον, ἢ, δύο, ,, ἢ, καθάπερ, τὰ, ὀνόματα, τρία, ,, τρία, καὶ, τὰ, γένη, διαιρούμενοι, καθʼ, ἓν, ὄνομα, γένος, ἑκάστῳ, προσῆπτον, ;, ΘΕΟ., ἀλλʼ, οὐδείς, ,, ὡς, ἐγᾦμαι, ,, φθόνος, αὐτῷ, διελθεῖν, αὐτά, ·, ἢ, πῶς, ,, ὦ, ξένε, ,, λέγωμεν, ;, ΞΕ., #, Ξένος, "", , ΞΕ., οὕτως, ,, ὦ, Θεόδωρε, ., φθόνος, μὲν, γὰρ, οὐδεὶς, οὐδὲ, χαλεπὸν, εἰπεῖν, ὅτι, γε, τρίʼ, ἡγοῦντο, ·, καθʼ, ἕκαστον, μὴν, διορίσασθαι, σαφῶς, τί, ποτʼ, ἔστιν, 

In [35]:
# convert conllu to spacy Lemmatization_training_files with NFKC normalization
!python -m spacy convert ../assets/Lemmatization_training_files/lemma_train_NFKC/ ../assets/Lemmatization_training_files/lemma_train_NFKC/ -c conllu -m --n-sents 10 --merge-subtokens

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (49 documents):
../assets/Lemmatization_training_files/lemma_train_NFKC/aeschylus_i_dev_NFKC.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (185 documents):
../assets/Lemmatization_training_files/lemma_train_NFKC/aeschylus_i_train_NFKC.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (33 documents):
../assets/Lemmatization_training_files/lemma_train_NFKC/aeschylus_ii_dev_NFKC.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (136 documents):
../assets/Lemmatization_training_files/lemma_train_NFKC/aeschylus_ii_train

In [36]:
#Normalize and convert Perseus dataset

import unicodedata
def normalize_NFKD(text):
    return unicodedata.normalize('NFKD', text)

# load files from folder
for file in os.listdir("../assets/UD_Ancient_Greek-Perseus"):
    if file.endswith(".conllu"):
        print(file)
        data = open("../assets/UD_Ancient_Greek-Perseus/" + file, "r", encoding="utf-8")
        annotations = data.read()
        sentences = conllu.parse(annotations)
        with open("../assets/UD_Ancient_Greek-Perseus/UD_Ancient_Greek-Perseus_NFKD/" + "{0}_NFKD.conllu".format(file[:-7]), "w", encoding="utf-8") as data:
            for sentence in sentences:
                for token in sentence:
                    token["form"] = normalize_NFKD(token["form"])
                    token["lemma"] = normalize_NFKD(token["lemma"])
                data.write(sentence.serialize())
        data.close()
        
import unicodedata
def normalize_NFKC(text):
    return unicodedata.normalize('NFKC', text)

# load files from folder
for file in os.listdir("../assets/UD_Ancient_Greek-Perseus"):
    if file.endswith(".conllu"):
        print(file)
        data = open("../assets/UD_Ancient_Greek-Perseus/" + file, "r", encoding="utf-8")
        annotations = data.read()
        sentences = conllu.parse(annotations)
        with open("../assets/UD_Ancient_Greek-Perseus/UD_Ancient_Greek-Perseus_NFKC/" + "{0}_NFKC.conllu".format(file[:-7]), "w", encoding="utf-8") as data:
            for sentence in sentences:
                for token in sentence:
                    token["form"] = normalize_NFKC(token["form"])
                    token["lemma"] = normalize_NFKC(token["lemma"])
                data.write(sentence.serialize())
        data.close()

grc_perseus-ud-train.conllu
grc_perseus-ud-test.conllu
grc_perseus-ud-dev.conllu
grc_perseus-ud-train.conllu
grc_perseus-ud-test.conllu
grc_perseus-ud-dev.conllu


In [37]:

#Normalize and convert PROEIL dataset

import unicodedata
def normalize_NFKD(text):
    return unicodedata.normalize('NFKD', text)

# Remove accents from the PROIEL files
# load files from folder
for file in os.listdir("../assets/UD_Ancient_Greek-PROIEL"):
    if file.endswith(".conllu"):
        print(file)
        data = open("../assets/UD_Ancient_Greek-PROIEL/" + file, "r", encoding="utf-8")
        annotations = data.read()
        sentences = conllu.parse(annotations)
        with open("../assets/UD_Ancient_Greek-PROIEL/UD_Ancient_Greek-PROIEL_NFKD/" + "{0}_NFKD.conllu".format(file[:-7]), "w", encoding="utf-8") as data:
            for sentence in sentences:
                for token in sentence:
                    token["form"] = normalize_NFKD(token["form"])
                    token["lemma"] = normalize_NFKD(token["lemma"])
                data.write(sentence.serialize())
        data.close()
        


import unicodedata
def normalize_NFKC(text):
    return unicodedata.normalize('NFKC', text)

# Remove accents from the PROIEL files
# load files from folder
for file in os.listdir("../assets/UD_Ancient_Greek-PROIEL"):
    if file.endswith(".conllu"):
        print(file)
        data = open("../assets/UD_Ancient_Greek-PROIEL/" + file, "r", encoding="utf-8")
        annotations = data.read()
        sentences = conllu.parse(annotations)
        with open("../assets/UD_Ancient_Greek-PROIEL/UD_Ancient_Greek-PROIEL_NFKC/" + "{0}_NKFC.conllu".format(file[:-7]), "w", encoding="utf-8") as data:
            for sentence in sentences:
                for token in sentence:
                    token["form"] = normalize_NFKC(token["form"])
                    token["lemma"] = normalize_NFKC(token["lemma"])
                data.write(sentence.serialize())
        data.close()
        

grc_proiel-ud-train.conllu
grc_proiel-ud-test.conllu
grc_proiel-ud-dev.conllu
grc_proiel-ud-train.conllu
grc_proiel-ud-test.conllu
grc_proiel-ud-dev.conllu


In [38]:
# import Universal dependencies tools validate.py


In [39]:
# convert conllu to spacy UD_Ancient_Greek and UD_Ancient_Greek-PROIEL
!python -m spacy convert ../assets/UD_Ancient_Greek-Perseus/UD_Ancient_Greek-Perseus_NFKD/ ../assets/UD_Ancient_Greek-Perseus/UD_Ancient_Greek-Perseus_NFKD/ -c conllu -m --n-sents 10 --merge-subtokens
!python -m spacy convert ../assets/UD_Ancient_Greek-Perseus/UD_Ancient_Greek-Perseus_NFKC/ ../assets/UD_Ancient_Greek-Perseus/UD_Ancient_Greek-Perseus_NFKC/ -c conllu -m --n-sents 10 --merge-subtokens

!python -m spacy convert ../assets/UD_Ancient_Greek-PROIEL/UD_Ancient_Greek-PROIEL_NFKD/ ../assets/UD_Ancient_Greek-PROIEL/UD_Ancient_Greek-PROIEL_NFKD/ -c conllu -m --n-sents 10 --merge-subtokens
!python -m spacy convert ../assets/UD_Ancient_Greek-PROIEL/UD_Ancient_Greek-PROIEL_NFKC/ ../assets/UD_Ancient_Greek-PROIEL/UD_Ancient_Greek-PROIEL_NFKC/ -c conllu -m --n-sents 10 --merge-subtokens



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (114 documents):
../assets/UD_Ancient_Greek-Perseus/UD_Ancient_Greek-Perseus_NFKD/grc_perseus-ud-dev_NFKD.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (131 documents):
../assets/UD_Ancient_Greek-Perseus/UD_Ancient_Greek-Perseus_NFKD/grc_perseus-ud-test_NFKD.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (1148 documents):
../assets/UD_Ancient_Greek-Perseus/UD_Ancient_Greek-Perseus_NFKD/grc_perseus-ud-train_NFKD.spacy
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism

In [ ]:
ord("̓")
# remove whitespace
testr = str(" ̓ ").strip()
print (testr)

In [ ]:
import spacy

In [ ]:
nlp = spacy.load("/root/Projects/Atlomy/git/greCy_ATLOMY/training/Lemmatize_transformer_morefiles/lemmatizer/model-best")

In [ ]:
# load spacy files
from spacy.tokens import DocBin, Doc

test_data= DocBin().from_disk("../corpus/train/lemma_train/aeschylus_train.spacy")
test_docbin_docs = list(test_data.get_docs(nlp.vocab))

In [ ]:
test_docbin_docs[0]

In [ ]:
# load spacy files
from spacy.tokens import DocBin, Doc

test_data= DocBin().from_disk("../assets/Lemmatization_training_files/lemma_train_clean/aeschylus_i_train.spacy")
test_docbin_docs = list(test_data.get_docs(nlp.vocab))

In [ ]:
test_docbin_docs[0]